In [ ]:

# Numpy 배열을 pandas DataFrame으로 변환하여 고유 클래스 수를 확인
df = pd.DataFrame(trn_dataset.df, columns=["name", "target"])
num_classes = df["target"].nunique()
class_counts = df['target'].value_counts().sort_index()
target_count = class_counts.max()


In [ ]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
import numpy as np
import os
from PIL import Image
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import os
import time
import timm
import torch
import albumentations as A
import pandas as pd
import numpy as np
import torch.nn as nn
import matplotlib.pyplot as plt
from albumentations.pytorch import ToTensorV2
from torch.optim import AdamW
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torch.optim.lr_scheduler import ReduceLROnPlateau
from PIL import Image
from tqdm import tqdm
from sklearn.metrics import accuracy_score, f1_score
import random


# 학습에 사용할 장치를 설정합니다.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 학습 설정
img_size = 224 
LR = 1e-3
EPOCHS = 5
BATCH_SIZE = 128
num_workers = 12  # 일단 0으로 설정하여 멀티프로세싱 비활성화
early_stopping_patience = 5  # Early Stopping 설정
augment_ratio_num = 100
transform_probablity = 0.9

model_path = 'tiny_vit_21m_384.dist_in22k_ft_in1k_384SIZE_16BATCH_5EPOCH_200_AUG_0.9_best_model.pth'
# 모델 로드
model_name = 'tiny_vit_21m_384.dist_in22k_ft_in1k'
model = timm.create_model(model_name, pretrained=False, num_classes=17).to(device)
model.load_state_dict(torch.load(model_path))


img_size = 384
# 기존 변환 설정
tst_transform = A.Compose([
    A.Resize(height=img_size, width=img_size),
    A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ToTensorV2(),
])

# TTA를 위한 변환 설정
tta_transforms = [
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.HorizontalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.VerticalFlip(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.RandomRotate90(p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.GaussianBlur(blur_limit=(3, 5), p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.MedianBlur(blur_limit=5, p=1.0),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.Downscale(scale_min=0.5, scale_max=0.75, p=1.0, interpolation=1),
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.RandomRotate90(p=0.8),
        A.MedianBlur(blur_limit=5, p=0.8),
        A.GaussianBlur(blur_limit=(3, 5), p=0.8),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
        A.VerticalFlip(p=0.8),
        A.HorizontalFlip(p=0.8),
        A.Rotate(limit=60, p=0.8),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.RandomRotate90(p=0.8),
        A.MedianBlur(blur_limit=5, p=0.8),
        A.GaussianBlur(blur_limit=(3, 5), p=0.8),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
        A.VerticalFlip(p=0.8),
        A.HorizontalFlip(p=0.8),
        A.Rotate(limit=60, p=0.8),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.RandomRotate90(p=0.8),
        A.MedianBlur(blur_limit=5, p=0.8),
        A.GaussianBlur(blur_limit=(3, 5), p=0.8),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
        A.VerticalFlip(p=0.8),
        A.HorizontalFlip(p=0.8),
        A.Rotate(limit=60, p=0.8),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
    A.Compose([
        A.Resize(height=img_size, width=img_size, interpolation=1),
        A.RandomRotate90(p=0.8),
        A.MedianBlur(blur_limit=5, p=0.8),
        A.GaussianBlur(blur_limit=(3, 5), p=0.8),
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
        A.VerticalFlip(p=0.8),
        A.HorizontalFlip(p=0.8),
        A.Rotate(limit=60, p=0.8),
        A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
        ToTensorV2(),
    ]),
]

# Dataset 정의
class ImageDataset(Dataset):
    def __init__(self, csv, path, transform=None, augment_ratios=None):
        self.df = pd.read_csv(csv)
        self.path = path
        self.transform = transform
        self.augment_ratios = augment_ratios
        self.indices = self._get_augmented_indices()

    def _get_augmented_indices(self):
        indices = []
        for i, row in self.df.iterrows():
            label = row['target']
            augment_count = self.augment_ratios.get(label, 1)
            indices.extend([i] * augment_count)
        return indices

    def __len__(self):
        return len(self.indices)

    def __getitem__(self, idx):
        real_idx = self.indices[idx]
        name, target = self.df.iloc[real_idx]
        img_path = os.path.join(self.path, name)
        
        try:
            img = np.array(Image.open(img_path))
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            img = np.zeros((img_size, img_size, 3), dtype=np.uint8)  # 빈 이미지로 대체
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# TTA를 적용한 예측을 수행하는 함수입니다.
def tta_predict(model, loader, tta_transforms, device):
    model.eval()
    all_preds = np.zeros((len(loader.dataset), 17), dtype=np.float32)

    for tta_transform in tta_transforms:
        tta_dataset = ImageDataset("datasets_fin/sample_submission.csv", "datasets_fin/test/", transform=tta_transform, augment_ratios={i: 1 for i in range(17)})
        tta_loader = DataLoader(
            tta_dataset,
            batch_size=loader.batch_size,
            shuffle=False,
            num_workers=loader.num_workers,
            pin_memory=loader.pin_memory
        )

        tta_preds = []
        for images, _ in tqdm(tta_loader):
            images = images.to(device)
            with torch.no_grad():
                preds = model(images)
            tta_preds.extend(preds.cpu().numpy())

        tta_preds = np.array(tta_preds)
        all_preds += tta_preds

    all_preds /= len(tta_transforms)  # 평균을 사용합니다.
    final_preds = np.argmax(all_preds, axis=1)  # 클래스 예측을 수행합니다.
    return final_preds

# 기존 데이터셋과 로더 정의
tst_dataset = ImageDataset(
    "datasets_fin/sample_submission.csv",
    "datasets_fin/test/",
    transform=tst_transform,
    augment_ratios={i: 1 for i in range(17)}
)

tst_loader = DataLoader(
    tst_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=num_workers,
    pin_memory=True
)

# TTA를 이용한 예측 수행
preds_list = tta_predict(model, tst_loader, tta_transforms, device)

# 예측 결과를 데이터프레임으로 저장합니다.
pred_df = pd.DataFrame(tst_dataset.df, columns=['ID', 'target'])
pred_df['target'] = preds_list

# 제출 형식 파일을 읽어와 ID 열이 일치하는지 확인합니다.
sample_submission_df = pd.read_csv("datasets_fin/sample_submission.csv")
assert (sample_submission_df['ID'] == pred_df['ID']).all()

# 예측 결과를 CSV 파일로 저장합니다.
# pred_df.to_csv(f"{model_name}_{img_size}SIZE_{BATCH_SIZE}BATCH_{EPOCHS}EPOCH_{augment_ratio_num}AUG_TST_pred.csv", index=False)
pred_df.to_csv(f"tiny_vit_21m_384.dist_in22k_ft_in1k_384SIZE_16BATCH_5EPOCH_200_AUG_0.9__TST_pred.csv", index=False)
print(pred_df.head())


In [7]:
import albumentations as A
from albumentations.pytorch import ToTensorV2
import pandas as pd
import numpy as np
import os
from PIL import Image
import timm
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

# 장치 설정
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 모델 정보
models_info = [
    {
        'path': 'efficientnet_b3.ra2_in1k_256SIZE_16BATCH_5EPOCH_200AUG_best_model.pth',
        'arch': 'efficientnet_b3.ra2_in1k',
        'img_size': 256
    },
    {
        'path': 'efficientnet_b5.sw_in12k_ft_in1k_384SIZE_16BATCH_5EPOCH_200_AUG_0.8_best_model.pth',
        'arch': 'efficientnet_b5.sw_in12k_ft_in1k',
        'img_size': 384
    },
    {
        'path': 'efficientnetv2_rw_m.agc_in1k_256SIZE_16BATCH_5EPOCH_200AUG_best_model.pth',
        'arch': 'efficientnetv2_rw_m.agc_in1k',
        'img_size': 256
    },
    {
        'path': 'tiny_vit_21m_384.dist_in22k_ft_in1k_384SIZE_16BATCH_5EPOCH_200_AUG_0.9_best_model.pth',
        'arch': 'tiny_vit_21m_384.dist_in22k_ft_in1k',
        'img_size': 384
    }
]

# Dataset 정의
class ImageDataset(Dataset):
    def __init__(self, df, path, transform=None):
        self.df = df
        self.path = path
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        name, target = self.df.iloc[idx]
        img_path = os.path.join(self.path, name)
        
        try:
            img = np.array(Image.open(img_path))
        except Exception as e:
            print(f"Error loading image {img_path}: {e}")
            img = np.zeros((img_size, img_size, 3), dtype=np.uint8)  # 빈 이미지로 대체
        
        if self.transform:
            img = self.transform(image=img)['image']
        return img, target

# TTA를 적용한 예측을 수행하는 함수입니다.
def tta_predict(model, loader, tta_transforms, device):
    model.eval()
    all_preds = np.zeros((len(loader.dataset), model.num_classes), dtype=np.float32)

    for tta_transform in tta_transforms:
        tta_dataset = ImageDataset(loader.dataset.df, loader.dataset.path, transform=tta_transform)
        tta_loader = DataLoader(
            tta_dataset,
            batch_size=loader.batch_size,
            shuffle=False,
            num_workers=loader.num_workers,
            pin_memory=loader.pin_memory
        )

        tta_preds = []
        for images, _ in tqdm(tta_loader):
            images = images.to(device)
            with torch.no_grad():
                preds = model(images)
            tta_preds.extend(preds.cpu().numpy())

        tta_preds = np.array(tta_preds)
        all_preds += tta_preds

    all_preds /= len(tta_transforms)  # 평균을 사용합니다.
    return all_preds

# 모델들의 예측 결과를 소프트 보팅으로 결합합니다.
def soft_voting(models_info, csv_path, img_path, batch_size, num_workers):
    final_preds = None
    df = pd.read_csv(csv_path)

    for model_info in models_info:
        # 모델 로드
        model = timm.create_model(model_info['arch'], pretrained=False, num_classes=17).to(device)
        model.load_state_dict(torch.load(model_info['path'], map_location=device))

        # TTA 변환 설정
        img_size = model_info['img_size']
        tta_transforms = [
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.HorizontalFlip(p=1.0),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.VerticalFlip(p=1.0),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.RandomRotate90(p=1.0),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.GaussianBlur(blur_limit=(3, 5), p=1.0),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=1.0),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.MedianBlur(blur_limit=5, p=1.0),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size),
                A.Downscale(scale_min=0.5, scale_max=0.75, p=1.0, interpolation=1),
                A.Resize(height=img_size, width=img_size),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size, interpolation=1),
                A.RandomRotate90(p=0.8),
                A.MedianBlur(blur_limit=5, p=0.8),
                A.GaussianBlur(blur_limit=(3, 5), p=0.8),
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
                A.VerticalFlip(p=0.8),
                A.HorizontalFlip(p=0.8),
                A.Rotate(limit=60, p=0.8),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size, interpolation=1),
                A.RandomRotate90(p=0.8),
                A.MedianBlur(blur_limit=5, p=0.8),
                A.GaussianBlur(blur_limit=(3, 5), p=0.8),
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
                A.VerticalFlip(p=0.8),
                A.HorizontalFlip(p=0.8),
                A.Rotate(limit=60, p=0.8),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size, interpolation=1),
                A.RandomRotate90(p=0.8),
                A.MedianBlur(blur_limit=5, p=0.8),
                A.GaussianBlur(blur_limit=(3, 5), p=0.8),
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
                A.VerticalFlip(p=0.8),
                A.HorizontalFlip(p=0.8),
                A.Rotate(limit=60, p=0.8),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
            A.Compose([
                A.Resize(height=img_size, width=img_size, interpolation=1),
                A.RandomRotate90(p=0.8),
                A.MedianBlur(blur_limit=5, p=0.8),
                A.GaussianBlur(blur_limit=(3, 5), p=0.8),
                A.Sharpen(alpha=(0.2, 0.5), lightness=(0.5, 1.0), p=0.8),
                A.VerticalFlip(p=0.8),
                A.HorizontalFlip(p=0.8),
                A.Rotate(limit=60, p=0.8),
                A.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
                ToTensorV2(),
            ]),
        ]

        # 데이터셋 및 로더 정의
        dataset = ImageDataset(df, img_path, transform=tta_transforms[0])
        loader = DataLoader(
            dataset,
            batch_size=batch_size,
            shuffle=False,
            num_workers=num_workers,
            pin_memory=True
        )

        # TTA 예측
        preds = tta_predict(model, loader, tta_transforms, device)

        if final_preds is None:
            final_preds = preds
        else:
            final_preds += preds

    final_preds /= len(models_info)  # 모델의 평균 예측을 사용합니다.
    final_preds = np.argmax(final_preds, axis=1)  # 클래스 예측을 수행합니다.
    return final_preds

# CSV 경로 및 이미지 경로
csv_path = "datasets_fin/sample_submission.csv"
img_path = "datasets_fin/test/"

# 소프트 보팅을 통한 최종 예측 수행
final_preds = soft_voting(models_info, csv_path, img_path, batch_size=16, num_workers=4)

# 예측 결과를 데이터프레임으로 저장합니다.
sample_submission_df = pd.read_csv(csv_path)
sample_submission_df['target'] = final_preds

# 예측 결과를 CSV 파일로 저장합니다.
sample_submission_df.to_csv("final_submission.csv", index=False)
print(sample_submission_df.head())


100%|██████████| 197/197 [00:11<00:00, 17.44it/s]

                     ID  target
0  0008fdb22ddce0ce.jpg       2
1  00091bffdffd83de.jpg      12
2  00396fbc1f6cc21d.jpg       5
3  00471f8038d9c4b6.jpg      12
4  00901f504008d884.jpg       2
